In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import datetime
import plotly.graph_objects as go
from ipywidgets import widgets, Layout
from IPython.display import display

In [2]:
import warnings
warnings.simplefilter(action="ignore",category=FutureWarning)

In [3]:
linkedin_job_postings=pd.read_csv("..\Data\linkedin_jobs_and_skills\linkedin_job_postings_cleaned.csv")

In [4]:
linkedin_job_postings.head()

,job_link,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,current_time,current_date
0,https://www.linkedin.com/jobs/view/account-exe...,account executive - dispensing (norcal/norther...,BD,"San Diego, CA",2024-01-15,Coronado,United States,Color Maker,Mid senior,Onsite,07:12:29.002560,2024-01-21
1,https://www.linkedin.com/jobs/view/registered-...,registered nurse - rn care manager,Trinity Health MI,"Norton Shores, MI",2024-01-14,Grand Haven,United States,Director Nursing Service,Mid senior,Onsite,07:39:58.881370,2024-01-21
2,https://www.linkedin.com/jobs/view/restaurant-...,restaurant supervisor - the forklift,Wasatch Adaptive Sports,"Sandy, UT",2024-01-14,Tooele,United States,Stand-In,Mid senior,Onsite,07:40:00.251126,2024-01-21
3,https://www.linkedin.com/jobs/view/independent...,independent real estate agent,Howard Hanna | Rand Realty,"Englewood Cliffs, NJ",2024-01-16,Pinehurst,United States,Real-Estate Clerk,Mid senior,Onsite,07:40:00.308133,2024-01-21
4,https://www.linkedin.com/jobs/view/group-unit-...,group/unit supervisor (systems support manager...,"IRS, Office of Chief Counsel","Chamblee, GA",2024-01-17,Gadsden,United States,Supervisor Travel-Information Center,Mid senior,Onsite,09:45:09.215838,2024-01-19


In [5]:
# Function to identify top hiring companies for a specific job position
def top_hiring_companies(jobs_df, top_n=10):
    
    # Count occurrences of each company
    company_counts = jobs_df['company'].value_counts().head(top_n)
    
    return company_counts

In [6]:
# Create initial plot
default_job_position = ''
initial_jobs_df = linkedin_job_postings[linkedin_job_postings['job_title'].str.contains(default_job_position.lower(), case=False)]
top_companies = top_hiring_companies(initial_jobs_df, 10)

In [50]:
fig = go.FigureWidget(
    data=[
        go.Bar(
            y=top_companies.index, 
            x=top_companies.values,
            orientation='h',
        )
    ]
).update_layout(
    title=f"Top Companies hiring",
    xaxis_title="Company",
    yaxis_title="Number of Job Postings",
    yaxis=dict(autorange="reversed"),
    height=500,
)

In [51]:
# Function to update plot based on filtered job position
def update_plot(search_value, top_n):
    if top_n < 1:
        top_n = 1
    if top_n > 20:
        top_n = 20
    if search_value:
        filtered_jobs_df = linkedin_job_postings[linkedin_job_postings['job_title'].str.contains(search_value.lower(), case=False)]
        if len(filtered_jobs_df) > 0:
            top_companies = top_hiring_companies(filtered_jobs_df, top_n)
            fig.data[0].y = [(i[:30] + '..') if len(i) > 30 else i for i in top_companies.index]
            fig.data[0].x = top_companies.values
            fig.update_layout(
                title=f"Top companies hiring for \'{search_value}\'",
                yaxis=dict(autorange="reversed"),
                height = 350 if (top_n < 7) else (50 * top_n),
            )
        else:
            fig.data[0].y = []
            fig.data[0].x = []
            fig.update_layout(title=f"No matching job positions for {search_value}")
    else:
        top_companies = top_hiring_companies(linkedin_job_postings, top_n)
        fig.data[0].y = [(i[:30] + '..') if len(i) > 30 else i for i in top_companies.index]
        fig.data[0].x = top_companies.values
        fig.update_layout(
            title=f"Top companies hiring",
            yaxis=dict(autorange="reversed"),
            height = 350 if (top_n < 7) else (50 * top_n),
        )

In [52]:
# Create search bar widget
search_widget = widgets.Text(
    value='',
    placeholder='Search for a job position...',
    description='Search:',
    continuous_update=False,
    layout=Layout(width='300px')
)

top_n_widget = widgets.IntSlider(
    value=10,
    description='Companies:',
    min=1,
    max=20,
    continuous_update=False,
    layout=Layout(width='400px')
)

# def update_plot_interactive(search_value, top_n):
#     return update_plot(search_value, top_n)

def on_search_button_clicked(b):
    update_plot(search_widget.value, top_n_widget.value)

In [53]:
# Create a button to trigger the search
search_button = widgets.Button(description="Search")

# Attach the button click event handler
search_button.on_click(on_search_button_clicked)

In [54]:
# Display search bar and initial plot
display(widgets.VBox([widgets.HBox([search_widget, top_n_widget, search_button]), fig]))